In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=False

In [2]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
        
def maskedarray_gt(data, value):
    """ Greater than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data > value] = value
    return data


def maskedarray_lt(data, value):
    """ Less than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data < value] = value
    return data


def ts_fromdebris_func(h, a, b, c):
    """ estimate surface temperature from debris thickness (h is debris thickness, a and k are coefficients) 
        Hill Equation"""
    return a * h**c / (b**c + h**c)


def debris_fromts_func(ts, a, b, c, hd_max=debris_prms.hd_max):
    """ estimate debris thickness from surface temperature (ts is surface temperature, a and k are coefficients) 
        Hill Equation"""
    # If temperature exceeds maximum of function cause NaN value
    max_value = ts_fromdebris_func(50, a, b, c)
    if ts.size == 1:
        if ts > max_value:
            ts = max_value
        if ts < 0:
            ts = 0
    else:
        ts[ts > a] = max_value
        ts[ts < 0] = 0
    # Debris thickness
    hd = (ts * b**c / (a - ts))**(1/c)
    return hd
        
    
def debris_fromts_maskedarray(ts_raw, a, b, c):
    """ Apply debris_fromts_func to masked array
        includes a mask of maximum values, since Michaelis-Mentin Equation has natural maximum 
    Parameters
    ----------
    ts_raw : np.ma.array
        masked array of the unmodified surface temperature
    Returns
    -------
    hd : np.ma.array 
        masked array of the debris thickness (m)
    """
    hd = debris_fromts_func(ts_raw.data, a, b, c)
    return hd

In [3]:
# ===== DETERMINE ALL GLACIERS WITH AND WITHOUT OBSERVATIONS =====
hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/'

# Glaciers optimized
glac_tsopt_fns = []
rgiid_list_tsopt = []
for roi_extrap in debris_prms.roi_dict_extrap[debris_prms.roi]:
    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_extrap + '/'
    for i in os.listdir(hdopt_prms_fp):
        if i.endswith('_hdopt_prms.csv'):
            region = int(i.split('.')[0])
            if region in debris_prms.roi_rgidict[roi_extrap]:    
                rgiid_list_tsopt.append(i.split('_')[0])            
                glac_tsopt_fns.append(i)
    glac_tsopt_fns = sorted(glac_tsopt_fns)
    rgiid_list_tsopt = sorted(rgiid_list_tsopt)

# if debris_prms.roi == '16':
#     hdopt_prms_fp2 = debris_prms.output_fp + 'hd_opt_prms/17/'
#     for i in os.listdir(hdopt_prms_fp2):
#         if i.endswith('_hdopt_prms.csv'):
#             region = int(i.split('.')[0])
#             if region in [17]:    
#                 rgiid_list_tsopt.append(i.split('_')[0])            
#                 glac_tsopt_fns.append(i)
#     glac_tsopt_fns = sorted(glac_tsopt_fns)
#     rgiid_list_tsopt = sorted(rgiid_list_tsopt)
    

main_glac_rgi_tsopt = debris_prms.selectglaciersrgitable(rgiid_list_tsopt)

# All debris-covered glaciers
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_rgiid = sorted([x.split('-')[1] for x in dc_shp.RGIId])
main_glac_rgi_all = debris_prms.selectglaciersrgitable(glac_no=dc_rgiid)

# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

# Add debris stats to area
dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area__1'].values))
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))

main_glac_rgi_tsopt['DC_Area_%'] = main_glac_rgi_tsopt.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_all['DC_Area_%'] = main_glac_rgi_all.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_tsopt['DC_Area_v2'] = main_glac_rgi_tsopt['Area'] * main_glac_rgi_tsopt['DC_Area_%'] / 100
main_glac_rgi_all['DC_Area_v2'] = main_glac_rgi_all['Area'] * main_glac_rgi_all['DC_Area_%'] / 100

# Glaciers lacking optimization
rgiids_missing = set(main_glac_rgi_all.rgino_str.values) - set(main_glac_rgi_tsopt.rgino_str.values)
rgiids_missing = sorted(rgiids_missing)
main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)
main_glac_rgi_missing

16 glaciers in region 16 are included in this model run: ['01963', '02137', '02172', '02322', '02348', '02394', '02410', '02412', '02420', '02433', '02457', '02482', '02483', '02497', '02518', '02566']
234 glaciers in region 17 are included in this model run: ['00516', '01019', '01448', '01506', '01687', '01917', '01940', '01972', '02025', '02427', '02600', '02698', '02737', '02806', '02857', '03289', '03291', '03301', '03309', '03311', '03318', '03325', '03348', '03353', '03359', '03371', '03406', '03418', '03424', '03425', '03451', '03455', '03469', '03519', '03531', '03545', '03610', '03645', '03646', '03655', '03750', '03899', '03903', '03925', '03927', '03929', '04065', '04101', '04108', '04110'] and more
This study is focusing on 250 glaciers in region [16, 17]
1433 glaciers in region 16 are included in this model run: ['00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00014', '00015', '00018', '00019', '00020', '00021', 

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,0,RGI60-16.00001,-69.9053,-16.521200,16,1,0.098,4896,5041,4966,21.4,181,348,0,0,9,20019999,1,16.00001,16.00001
1,1,RGI60-16.00002,-69.9117,-16.519600,16,1,0.147,4963,5056,5005,20.7,169,289,0,0,9,20019999,2,16.00002,16.00002
2,2,RGI60-16.00003,-69.9246,-16.515600,16,1,0.149,4892,5043,4963,25.4,176,337,0,0,9,20019999,3,16.00003,16.00003
3,3,RGI60-16.00004,-69.1445,-16.385500,16,1,0.051,4443,4521,4491,15.4,185,320,0,0,9,20019999,4,16.00004,16.00004
4,4,RGI60-16.00005,-69.1411,-16.384000,16,1,0.037,4397,4522,4453,20.7,144,347,0,0,9,20019999,5,16.00005,16.00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,2919,RGI60-16.02926,-76.4684,-11.300100,16,1,0.143,4921,5258,5133,25.8,157,753,0,0,9,20060528,2926,16.02926,16.02926
1413,2921,RGI60-16.02928,-76.4924,-11.199000,16,1,0.029,4965,5027,5017,10.9,183,374,0,0,9,20060528,2928,16.02928,16.02928
1414,2935,RGI60-16.02942,-78.4438,-0.697071,16,1,1.564,4703,5818,5134,28.1,210,2130,0,0,9,19979999,2942,16.02942,16.02942


In [4]:
print('\n', debris_prms.roi + ': calibration includes', main_glac_rgi_tsopt.shape[0], 'glaciers covering',
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum(),1)), 'km2 (' + 
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum() / main_glac_rgi_all['DC_Area_v2'].sum() * 100,1)) + 
      '%) of the total debris-covered glacier area\n')     


 16: calibration includes 250 glaciers covering 13.8 km2 (3.9%) of the total debris-covered glacier area



In [5]:
# # print('\nHACK TO EXTRAPOLATE TO CALIBRATED GLACIERS FOR COMPARISON\n')
# rgiids_missing = ['15.02167']
# # # rgiids_missing = ['2.14297']
# # # rgiids_missing = ['11.02472']
# # # rgiids_missing = ['12.01132']
# # # rgiids_missing = ['13.43165']
# # # rgiids_missing = ['15.03473']
# # # rgiids_missing = ['15.04045']
# # # rgiids_missing = ['18.02397']
# # rgiids_missing = ['11.03206']

# main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)

In [6]:
extrap_uncalibrated_glaciers = True
overwrite_hd = False

hd_fp = debris_prms.hd_fp + 'extrap/'
if not os.path.exists(hd_fp):
    os.makedirs(hd_fp)
    
mf_fp = hd_fp + 'meltfactor/'
if not os.path.exists(mf_fp):
    os.makedirs(mf_fp)
    
fig_extrap = debris_prms.output_fig_fp + debris_prms.roi + '/' + 'extrap/'
if not os.path.exists(fig_extrap):
    os.makedirs(fig_extrap)

if extrap_uncalibrated_glaciers:
    
    # ===== NEAREST GLACIERS WITH DATA =====
    n_glac_nearest = 1000
    if n_glac_nearest > main_glac_rgi_tsopt.shape[0]:
        n_glac_nearest = main_glac_rgi_tsopt.shape[0]

    nearest_dict = {}
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[755]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        if glac_idx%500 == 0:
            print(glac_idx, glac_str)
        latlon_dist = (((main_glac_rgi_tsopt['CenLat'].values - main_glac_rgi_missing['CenLat'].values[glac_idx])**2 + 
                        (main_glac_rgi_tsopt['CenLon'].values - main_glac_rgi_missing['CenLon'].values[glac_idx])**2)**0.5)

        latlon_nearidx_list = np.argsort(latlon_dist)[0:n_glac_nearest]
        rgiid_nearest_list = list(main_glac_rgi_tsopt.loc[latlon_nearidx_list,'rgino_str'].values)

        nearest_dict[glac_str] = rgiid_nearest_list

    # Ts filenames
    ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values[150:200]):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[0]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        rgiid = main_glac_rgi_missing.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        # Ts filename
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        
        # Hd filename
        hd_fn = debris_prms.hd_fn_sample.replace('XXXX', glac_str_noleadzero).replace('.tif','_extrap.tif')

        # Ice thickness filenames
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if ((not os.path.exists(hd_fp + hd_fn) or overwrite_hd) and os.path.exists(thick_dir + thick_fn) and 
            ts_fn not in ['0.0']):
            
            print(glac_idx, glac_str)
            
            # Create glacier feature
            gf = create_glacfeat(thick_dir, thick_fn)
        
            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if not os.path.exists(dc_shp_proj_fn):
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + 
                                            debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + 
                                                    str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_ts=True, ts_fn=ts_fn, gf_add_slope_aspect=False)

#             # ===== PLOTS =====
#             if debug:
#                 # DEM
#                 var_full2plot = gf.z1.copy()
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#                 # Surface temperature
#                 var_full2plot = gf.ts.copy()
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#                 # Surface temperature (debris-covered)
#                 var_full2plot = gf.ts.copy()
#                 var_full2plot.mask = gf.dc_mask
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
            

            # ===== SURFACE TEMPERATURE FOR THINNEST DEBRIS (connected to terminus) =====
            outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)   
            bin_idx_dc = np.where(outbins_df['dc_bin_count_valid'] > 0)[0]
            bin_idx_dif = list(bin_idx_dc[1:] - bin_idx_dc[:-1])
            if not(np.sum(bin_idx_dif) == len(bin_idx_dc)-1):
                idx_jumpinbins = bin_idx_dif.index(next(filter(lambda x: x>1, bin_idx_dif)))
                bin_idx_dc = bin_idx_dc[0:idx_jumpinbins+1]
            
            ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            
            if debug:
                print('ts_min:', np.round(ts_min,1), ' ts_max:', np.round(ts_max,1))
            
            if np.isnan(ts_min) and np.isnan(ts_max):
                troubleshoot_fp = (debris_prms.output_fp + 'errors/no_Ts_data-extrap/' + debris_prms.roi + '/')
                if not os.path.exists(troubleshoot_fp):
                    os.makedirs(troubleshoot_fp)
                txt_fn_extrapfail = glac_str + "-noTs-extrap.txt"
                with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                    text_file.write(glac_str + ' no surface temperature data but made it past preprocessing to extrap')

            else:
                # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                # Load parameters from nearest neighbor
                rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                n_nearest = 0
                n_success = 0
                min_n_nearest = 10
                hd_ts_list = []
                mf_list = []
                while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                    rgi_str_nearest = rgiid_nearest_list[n_nearest]
                    if rgi_str_nearest.startswith('0'):
                        rgi_str_nearest = rgi_str_nearest[1:]
#                     if debug:
#                         print(n_nearest, 'rgi nearest:', rgi_str_nearest)

                    # Load parameters
                    df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                    roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                    if roi_nearest in ['13','14','15']:
                        roi_nearest = 'HMA'
                    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                    df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                    melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                    melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                    func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                    func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                    # Estimate debris thickness of thinnest bin
                    hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    
                    if debug:
                        print('     thin:', np.round(ts_min,1), np.round(hd_thin,3), 'm', 
                              '    thick:', np.round(ts_max,1), np.round(hd_thick,3), 'm')

    #                 print(func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2], ts_min, hd_thin)

                    # Minimum and maximum debris thickness are reasonable
                    if hd_thin > 0 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                        if debug:
                            print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                        hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                        hd_array[hd_array<0] = 0

                        hd_ts_list.append(hd_array)
                        n_success += 1

                    n_nearest += 1
                    
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd1/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd1.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in first round')
                    
                    
                # ===== SECOND ROUND: NEGATIVE VALUES CAUSE HD_THIN TO BE 0 =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_min < 0:
                    
                    if debug:
                        print('\n-----\nROUND 2 OF EXTRAPOLATION')
                    
                    ts_offset = abs(ts_min)
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
#                             ts_array = gf.ts.data.copy() + ts_offset
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                            if roi_nearest in ['13','14','15']:
                                roi_nearest = 'HMA'
                            hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1
                        ts_offset = 1
                        
                        
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd2/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd2.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in second round')
                    
                # ===== THIRD ROUND: ASSUME TOO POSITIVE CAUSING HD TO BE VERY THICK =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_max > 20:
                    
                    if debug:
                        print('\n-----\nROUND 3 OF EXTRAPOLATION')
                    
                    ts_offset = -1
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                            if roi_nearest in ['13','14','15']:
                                roi_nearest = 'HMA'
                            hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1

                # ===== ESTIMATE DEBRIS THICKNESS FROM ALL COMBINATIONS =====
                if len(hd_ts_list) > 0:
                    # DEBRIS THICKNESS based on median of the plausible nearest values
                    hd_ts_all = np.array(hd_ts_list)
                    hd_ts_med = np.median(hd_ts_all, axis=0)
                    gf.debris_thick_ts = np.ma.array(hd_ts_med, mask=gf.dc_mask)

                    if debug:
                        close_fig=False
                    else:
                        close_fig=True

                    # Debris thickness
                    var_full2plot = gf.debris_thick_ts.copy()
                    clim = (0,1)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' hd (from ts)'], 'inferno', 'hd (m)', 
                               fn=fig_extrap + gf.feat_fn +'_hd_ts.png', close_fig=close_fig)

                    # ===== EXPORT DEBRIS THICKNESS AND MELT FACTOR TIFS ===== 
                    # Debris thickness
                    gf.debris_thick_ts.mask = gf.dc_mask
                    iolib.writeGTiff(gf.debris_thick_ts, hd_fp + hd_fn, gf.ds_dict['z1'])

                    # Melt factor
                    gf.meltfactor_ts = (
                        melt_fromdebris_func(gf.debris_thick_ts, func_coeff[0], func_coeff[1]) / melt_cleanice)
                    # limit melt rates to modeled 2 cm rate
                    gf.meltfactor_ts = np.ma.array(
                        maskedarray_gt(gf.meltfactor_ts, melt_2cm / melt_cleanice), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Linearly interpolate between 0 cm and 2 cm for the melt rate
                    def meltfactor_0to2cm_adjustment(mf, melt_clean, melt_2cm, hd):
                        """ Linearly interpolate melt factors between 0 and 2 cm 
                            based on clean ice and 2 cm sub-debris melt """
                        mf = np.nan_to_num(mf,0)
                        mf[(hd >= 0) & (hd < 0.02)] = (
                            1 + hd[(hd >= 0) & (hd < 0.02)] / 0.02 * (melt_2cm - melt_clean) / melt_clean)
                        return mf
                    gf.meltfactor_ts = np.ma.array(
                        meltfactor_0to2cm_adjustment(gf.meltfactor_ts.data.copy(), melt_cleanice, melt_2cm, 
                                                     gf.debris_thick_ts.data), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Plot melt factor
                    var_full2plot = gf.meltfactor_ts.copy()
                    clim = (0,1.25)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' melt factor'], 'inferno', 'melt factor (-)', 
                               fn=fig_extrap + gf.feat_fn +'_mf.png', close_fig=True)

                    gf.meltfactor_ts.mask = gf.dc_mask
                    mf_fn = debris_prms.mf_fn_sample.replace('XXXX',gf.glacnum).replace('.tif','_extrap.tif')
                    iolib.writeGTiff(gf.meltfactor_ts, mf_fp + mf_fn, gf.ds_dict['z1'])


                    # ===== EXPORT THE BINNED DEBRIS THICKNESS AND MELT FACTOR =====
                    # Output debris thickness
                    outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                    hd_extrap_bin_fp = debris_prms.mb_binned_fp_wdebris_hdts + '../_wdebris_hdts_extrap/'
                    if not os.path.exists(hd_extrap_bin_fp):
                        os.makedirs(hd_extrap_bin_fp)
                    outbins_df.to_csv(hd_extrap_bin_fp + glac_str + '_mb_bins_hdts_extrap.csv', index=False)

                else:
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates')

0 16.00001
500 16.00625
1000 16.01228
0 16.00001
1 16.00002
2 16.00003
3 16.00004
4 16.00005
5 16.00006
6 16.00007
7 16.00008
8 16.00009
9 16.00010
10 16.00011
11 16.00012
12 16.00014
13 16.00015
14 16.00018
15 16.00019
16 16.00020
17 16.00021
18 16.00022
19 16.00023
20 16.00025
21 16.00026
22 16.00027
23 16.00028
24 16.00029
25 16.00030
26 16.00031
27 16.00032
28 16.00033
29 16.00034
30 16.00035
31 16.00036
32 16.00037
33 16.00038
34 16.00039
35 16.00040
36 16.00041
37 16.00042
38 16.00043
39 16.00044
40 16.00045
41 16.00046
42 16.00047
43 16.00048
44 16.00049
45 16.00050
46 16.00051
47 16.00052
48 16.00054
49 16.00056
50 16.00058
51 16.00059
52 16.00060
53 16.00062
54 16.00063
55 16.00064
56 16.00066
57 16.00067
58 16.00068
59 16.00069
60 16.00070
61 16.00071
62 16.00072
63 16.00074
64 16.00075
65 16.00078
66 16.00079
67 16.00080
68 16.00082
69 16.00084
70 16.00085
71 16.00086
72 16.00087
73 16.00090
74 16.00091
75 16.00092
76 16.00094
77 16.00095
78 16.00096
79 16.00098
80 16.00099


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:113: RuntimeWarning: All-NaN slice encountered
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:114: RuntimeWarning: All-NaN slice encountered


169 16.00219
170 16.00220
171 16.00221
172 16.00222
173 16.00223
174 16.00225
175 16.00226
176 16.00227
177 16.00228
178 16.00229
179 16.00230
180 16.00231
181 16.00233
182 16.00234
183 16.00235
184 16.00236
185 16.00237
186 16.00238
187 16.00239
188 16.00240
189 16.00241
190 16.00243
191 16.00244
192 16.00245
193 16.00246
194 16.00247
195 16.00248
196 16.00249
197 16.00251
198 16.00253
199 16.00254
200 16.00256
201 16.00257
202 16.00258
203 16.00259
204 16.00260
205 16.00261
206 16.00263
207 16.00264
208 16.00265
209 16.00266
210 16.00267
211 16.00268
212 16.00269
213 16.00270
214 16.00272
215 16.00273
216 16.00274
217 16.00275
218 16.00276
219 16.00277
220 16.00278
221 16.00279
222 16.00280
223 16.00281
224 16.00282
225 16.00285
226 16.00286
227 16.00287
228 16.00288
229 16.00289
230 16.00290
231 16.00291
232 16.00292
233 16.00293
234 16.00294
235 16.00295
236 16.00296
237 16.00297
238 16.00298
239 16.00299
240 16.00300
241 16.00301
242 16.00302
243 16.00304
244 16.00305
245 16.00306

800 16.00975
801 16.00976
802 16.00977
803 16.00978
804 16.00979
805 16.00980
806 16.00981
807 16.00982
808 16.00983
809 16.00984
810 16.00986
811 16.00987
812 16.00988
813 16.00994
814 16.00996
815 16.00997
816 16.00998
817 16.01000
818 16.01001
819 16.01002
820 16.01003
821 16.01004
822 16.01005
823 16.01006
824 16.01007
825 16.01008
826 16.01014
827 16.01015
828 16.01016
829 16.01017
830 16.01018
831 16.01019
832 16.01020
833 16.01021
834 16.01022
835 16.01023
836 16.01024
837 16.01025
838 16.01026
839 16.01027
840 16.01028
841 16.01029
842 16.01030
843 16.01031
844 16.01032
845 16.01034
846 16.01035
847 16.01036
848 16.01037
849 16.01038
850 16.01039
851 16.01040
852 16.01042
853 16.01043
854 16.01044
855 16.01045
856 16.01046
857 16.01047
858 16.01048
859 16.01050
860 16.01051
861 16.01052
862 16.01053
863 16.01054
864 16.01055
865 16.01057
866 16.01058
867 16.01059
868 16.01060
869 16.01061
870 16.01062
871 16.01063
872 16.01064
873 16.01065
874 16.01066
875 16.01067
876 16.01068

1402 16.02908
1403 16.02911
1404 16.02914
1405 16.02916
1406 16.02917
1407 16.02918
1408 16.02919
1409 16.02921
1410 16.02923
1411 16.02924
1412 16.02926
1413 16.02928
1414 16.02942
1415 16.02944
1416 16.02945


In [7]:
# bin_idx_dc_thin = bin_idx_dc[-1]
# outbins_df.loc[bin_idx_dc_thin, 'hd_ts_med_m']
# outbins_df.loc[:,['bin_center_elev_m', 'dc_ts_med', 'hd_ts_med_m']]

In [8]:
print('\nDONE!\n')


DONE!



In [9]:
print('TO-DO LIST:')
print('  - Mosaic extrapolated and calibrated glaciers into 1-degree pixels')
print('  - Test robustness on glaciers with data (Ngozumpa, Miage, etc.)')

TO-DO LIST:
  - Mosaic extrapolated and calibrated glaciers into 1-degree pixels
  - Test robustness on glaciers with data (Ngozumpa, Miage, etc.)
